In [25]:
import os

import dgl
import numpy as np
import pandas as pd
import torch
import tqdm
import os

In [3]:
class SAGE(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.ModuleList()
        self.layers.append(dgl.nn.SAGEConv(602, 320, 'mean'))
        self.layers.append(dgl.nn.SAGEConv(320, 160, 'mean'))
        self.layers.append(torch.nn.Sequential(
            torch.nn.Linear(160, 80),
            torch.nn.ReLU(),
            torch.nn.Linear(80, 41),
            torch.nn.Softmax()
        ))

    def forward(self, g, x):
        x_new = x
        for i, g_i in enumerate(g):
            x_new = self.layers[i](g_i, x_new)
        return self.layers[-1](x_new)


In [17]:
dataset = dgl.data.RedditDataset()
graph = dataset[0]

In [33]:
model = SAGE()
sampler = dgl.dataloading.NeighborSampler([20, 20])
train_dataloader = dgl.dataloading.DataLoader(
    # The following arguments are specific to DGL's DataLoader.
    graph,              # The graph
    graph.nodes()[graph.ndata["train_mask"]],         # The node IDs to iterate over in minibatches
    sampler,            # The neighbor sampler
    # The following arguments are inherited from PyTorch DataLoader.
    batch_size=1024,    # Batch size
    shuffle=True,       # Whether to shuffle the nodes for every epoch
)
opt = torch.optim.Adam(model.parameters())
loss = torch.nn.CrossEntropyLoss()

In [34]:
EPOCHS = 10
for i in range(EPOCHS):
    with tqdm.tqdm(train_dataloader) as tq:
        for step, (input_nodes, output_nodes, mfgs) in enumerate(tq):
            # feature copy from CPU to GPU takes place here
            inputs = mfgs[0].srcdata['feat']
            labels = mfgs[-1].dstdata['label']
            predictions = model(mfgs, inputs)
            loss_v = loss(predictions, labels)
            opt.zero_grad()
            loss_v.backward()
            opt.step()
            accuracy = torch.sum(predictions.max(dim=1)[1] == labels) / labels.shape[0]

            tq.set_postfix({'loss': '%.03f' % loss_v.item(), 'accuracy': "%.03f" % accuracy}, refresh=False)

  0%|          | 0/150 [00:00<?, ?it/s]/home/rustambaku13/miniconda3/envs/ml/lib/python3.8/site-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
 63%|██████▎   | 95/150 [01:25<00:49,  1.10it/s, loss=3.067, accuracy=0.689]


KeyboardInterrupt: 